In [ ]:
import pandas as pd
import joblib
import os
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report, precision_score
from xgboost import XGBClassifier

FILENAME = 'minesweeper_dataset.csv'
MODEL_NAME = 'minesweeper_ai_model.pkl'

# 1. Carica Dati
if not os.path.exists(FILENAME):
    print(f"ERRORE: {FILENAME} non trovato.")
    exit()

print("Caricamento dataset...")
df = pd.read_csv(FILENAME)
df = df.drop_duplicates()
print(f"Righe uniche: {len(df)}")

X = df.drop('safe', axis=1)
y = df['safe']

# XGBoost vuole nomi di colonne senza caratteri speciali a volte, 
# ma il tuo csv ha 'cell_-1_-1' che va bene.

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# 2. Configurazione XGBoost
# Nota: XGBoost è molto potente, limitiamo la depth per evitare overfitting
model = XGBClassifier(
    n_estimators=200,       # Numero di round di correzione
    learning_rate=0.05,     # Passo piccolo = impara lentamente ma meglio
    max_depth=6,            # Gli alberi di XGBoost sono solitamente più bassi di RF
    min_child_weight=1,     # Simile a min_samples_leaf
    subsample=0.8,          # Usa solo l'80% dei dati per ogni albero (evita overfitting)
    colsample_bytree=0.8,   # Usa solo l'80% delle feature per albero
    n_jobs=-1,              # Usa tutta la CPU
    random_state=42,
    eval_metric='logloss',  # Metrica di errore
    use_label_encoder=False
)

print("Addestramento XGBoost in corso...")
model.fit(X_train, y_train)

# 3. Valuta
predictions = model.predict(X_test)
precision = precision_score(y_test, predictions, pos_label=1)
accuracy = accuracy_score(y_test, predictions)

print("-" * 40)
print(f"XGBOOST RESULTS")
print("-" * 40)
print(f"ACCURACY:       {accuracy * 100:.2f}%")
print(f"PRECISION SAFE: {precision * 100:.2f}%") # <--- Questo è il numero che ti interessa
print("-" * 40)
print(classification_report(y_test, predictions))

# 4. Salva
joblib.dump(model, MODEL_NAME)
print(f"Modello salvato: {MODEL_NAME}")

Caricamento dataset...
Righe uniche: 287838
Addestramento XGBoost in corso...
----------------------------------------
XGBOOST RESULTS
----------------------------------------
ACCURACY:       85.76%
PRECISION SAFE: 86.07%
----------------------------------------
              precision    recall  f1-score   support

           0       0.50      0.03      0.06      8193
           1       0.86      0.99      0.92     49375

    accuracy                           0.86     57568
   macro avg       0.68      0.51      0.49     57568
weighted avg       0.81      0.86      0.80     57568

Modello salvato: minesweeper_ai_model.pkl
